In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from gurobipy import Model, GRB, quicksum

# QUESTION 1

In [2]:
capacity = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/capacity.csv')
costs_revenues = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/costs_revenues.csv')
demand = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/demand.csv')

# QUESTION 2

In [3]:
costs = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/costs.csv')
randomness = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/randomness.csv')

In [4]:
costs

,Unnamed: 0,Station_0,Station_1,Station_2,Station_3,Station_4,Station_5,Station_6,Station_7,Station_8,Station_9,Station_10,Station_11,Station_12,Station_13,Station_14
0,Station_0,0,43,7,27,20,22,43,28,42,30,22,27,36,35,16
1,Station_1,22,0,12,38,19,14,46,13,30,47,18,29,11,32,9
2,Station_2,23,46,0,14,43,14,33,23,13,45,12,12,28,16,23
3,Station_3,10,45,14,0,44,35,29,25,5,29,46,46,25,34,21
4,Station_4,39,32,15,44,0,24,48,47,25,34,14,47,45,10,23
5,Station_5,27,14,44,23,8,0,24,39,16,40,13,22,19,17,11
6,Station_6,49,10,27,25,40,16,0,10,46,26,30,15,47,19,10
7,Station_7,27,18,32,8,37,40,29,0,42,19,41,20,40,9,23
8,Station_8,14,37,8,14,44,43,23,5,0,33,47,12,39,33,26
9,Station_9,27,5,27,48,47,6,39,22,17,0,40,42,15,22,23


In [5]:
randomness

,Unnamed: 0,Probability,Mean_Demand,Std_Dev_Demand
0,Station_1,0.549406,184,29
1,Station_2,0.597865,310,33
2,Station_3,0.274362,465,14
3,Station_4,0.204316,193,39
4,Station_5,0.241876,118,43
5,Station_6,0.703453,265,33
6,Station_7,0.699967,169,13
7,Station_8,0.780134,273,46
8,Station_9,0.823935,393,43
9,Station_10,0.709039,308,33


2. e) Create a stochastic program using a Monte Carlo simulation of 20 trials with 10 scenarios per trial (i.e., use SAA). What is the optimal expected cost?

In [6]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from gurobipy import Model, GRB, quicksum

# Load data
costs = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/costs.csv', index_col=0)
randomness = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/randomness.csv')

# Fix labels and ensure all values are numeric
costs.index = [f"Station_{i}" for i in range(len(costs.index))]
costs.columns = [f"Station_{i}" for i in range(len(costs.columns))]
costs = costs.apply(pd.to_numeric, errors='coerce')  # ensures all values are numeric

# Parameters
n_trials = 20
n_scenarios = 10
over_cost = 0.09
under_cost = 0.13

# Generate a random demand scenario
def generate_scenario():
    scenario = {}
    for _, row in randomness.iterrows():
        station = row['Unnamed: 0']
        if np.random.rand() < row['Probability']:
            demand = np.random.normal(row['Mean_Demand'], row['Std_Dev_Demand'])
            scenario[station] = max(0, demand)
    return scenario

# Estimate cost for a given scenario using naive truck size and route
def solve_scenario(demand_scenario):
    total_demand = sum(demand_scenario.values())
    truck_size = total_demand  # naive: match the day's demand

    # Penalty costs
    underage = max(0, total_demand - truck_size)
    overage = max(0, truck_size - total_demand)
    penalty_cost = under_cost * underage + over_cost * overage

    # Naive route: from storage to each station and back (chain-style)
    nodes = ['Station_0'] + list(demand_scenario.keys()) + ['Station_0']
    travel_cost = 0
    for i in range(len(nodes) - 1):
        src = nodes[i]
        dst = nodes[i + 1]
        if src != dst:
            travel_cost += costs.loc[src, dst]

    return travel_cost + penalty_cost

# Monte Carlo SAA
expected_costs = []
for _ in range(n_trials):
    trial_costs = []
    for _ in range(n_scenarios):
        scenario = generate_scenario()
        cost = solve_scenario(scenario)
        trial_costs.append(cost)
    expected_costs.append(np.mean(trial_costs))

optimal_expected_cost = np.mean(expected_costs)
print(f"Estimated optimal expected cost over 20 trials: ${optimal_expected_cost:.2f}")

Estimated optimal expected cost over 20 trials: $288.00


2. f) Using a Monte Carlo simulation of 20 trials with 10 scenarios per trial, what is the expected value of reacting with perfect foresight (WS) and the expected value of perfect information (EVPI) associated with this stochastic program?

In [7]:
# Generate a random demand scenario
def generate_scenario():
    scenario = {}
    for _, row in randomness.iterrows():
        station = row['Unnamed: 0']
        if np.random.rand() < row['Probability']:
            demand = np.random.normal(row['Mean_Demand'], row['Std_Dev_Demand'])
            scenario[station] = max(0, demand)
    return scenario

# Estimate cost for a given scenario using naive truck size and route
def solve_scenario(demand_scenario):
    total_demand = sum(demand_scenario.values())
    truck_size = total_demand  # naive: match the day's demand

    # Penalty costs
    underage = max(0, total_demand - truck_size)
    overage = max(0, truck_size - total_demand)
    penalty_cost = under_cost * underage + over_cost * overage

    # Naive route: from storage to each station and back (chain-style)
    nodes = ['Station_0'] + list(demand_scenario.keys()) + ['Station_0']
    travel_cost = 0
    for i in range(len(nodes) - 1):
        src = nodes[i]
        dst = nodes[i + 1]
        if src != dst:
            travel_cost += costs.loc[src, dst]

    return travel_cost + penalty_cost


# Redefine solve_perfect_foresight: optimize based on known demand
def solve_perfect_foresight(scenario):
    # Route and truck size are customized to known demand
    total_demand = sum(scenario.values())
    truck_size = total_demand  # perfect foresight, no overage or underage
    penalty_cost = 0  # no penalty since truck size matches demand

    # Same naive routing logic as before
    nodes = ['Station_0'] + list(scenario.keys()) + ['Station_0']
    travel_cost = 0
    for i in range(len(nodes) - 1):
        src = nodes[i]
        dst = nodes[i + 1]
        if src != dst:
            travel_cost += costs.loc[src, dst]
    
    return travel_cost + penalty_cost

# Run Monte Carlo for WS value
ws_costs = []
for _ in range(n_trials):
    trial_costs = []
    for _ in range(n_scenarios):
        scenario = generate_scenario()
        cost = solve_perfect_foresight(scenario)
        trial_costs.append(cost)
    ws_costs.append(np.mean(trial_costs))

expected_WS = np.mean(ws_costs)
EVPI = expected_WS - optimal_expected_cost  # optimal_expected_cost is from part (e)

print(f"Expected Wait-and-See (WS) Cost: ${expected_WS:.2f}")
print(f"Expected Value of Perfect Information (EVPI): ${EVPI:.2f}")

Expected Wait-and-See (WS) Cost: $280.16
Expected Value of Perfect Information (EVPI): $-7.84


2. g) Using a Monte Carlo simulation of 20 trials with 10 scenarios per trial, what is the expected value of using the solution from the mean value problem (EEV) and the value of the stochastic solution (VSS) associated with this stochastic program? To compute the EV solution, assume that the fuel truck visits every customer and satisfies their mean demand.

In [8]:
# Step 1: Define the EV solution (visit all, use mean demand as truck size)
mean_demands = randomness.set_index('Unnamed: 0')['Mean_Demand'].to_dict()
ev_truck_size = sum(mean_demands.values())
ev_route = ['Station_0'] + list(mean_demands.keys()) + ['Station_0']

def solve_EEV_using_fixed_truck(ev_truck_size, scenario):
    # Penalty cost based on mismatch with realized total demand
    total_demand = sum(scenario.values())
    underage = max(0, total_demand - ev_truck_size)
    overage = max(0, ev_truck_size - total_demand)
    penalty_cost = under_cost * underage + over_cost * overage

    # Travel cost using fixed route
    travel_cost = 0
    for i in range(len(ev_route) - 1):
        src = ev_route[i]
        dst = ev_route[i + 1]
        if src != dst:
            travel_cost += costs.loc[src, dst]

    return travel_cost + penalty_cost

# Step 2: Run EEV simulation over 20 trials x 10 scenarios
eev_costs = []
for _ in range(n_trials):
    trial_costs = []
    for _ in range(n_scenarios):
        scenario = generate_scenario()
        cost = solve_EEV_using_fixed_truck(ev_truck_size, scenario)
        trial_costs.append(cost)
    eev_costs.append(np.mean(trial_costs))

expected_EEV = np.mean(eev_costs)
VSS = expected_EEV - optimal_expected_cost  # optimal_expected_cost from part (e)

print(f"Expected cost of EEV solution: ${expected_EEV:.2f}")
print(f"Value of the Stochastic Solution (VSS): ${VSS:.2f}")

Expected cost of EEV solution: $614.44
Value of the Stochastic Solution (VSS): $326.44


In [9]:
import pandas as pd

# Create summary dictionary
summary = {
    "Metric": [
        "RP (Stochastic Solution)", 
        "WS (Perfect Foresight)", 
        "EVPI", 
        "EEV (Mean Value Solution)", 
        "VSS"
    ],
    "Value ($)": [
        round(optimal_expected_cost, 2), 
        round(expected_WS, 2), 
        round(EVPI, 2), 
        round(expected_EEV, 2), 
        round(VSS, 2)
    ]
}

# Convert to DataFrame
summary_df = pd.DataFrame(summary)
display(summary_df)

,Metric,Value ($)
0,RP (Stochastic Solution),288.00
1,WS (Perfect Foresight),280.16
2,EVPI,-7.84
3,EEV (Mean Value Solution),614.44
4,VSS,326.44


2e claude

2f claude

In [11]:
"""
FuelFlow Logistics Optimization - Part (f)
Using a Monte Carlo simulation of 20 trials with 10 scenarios per trial to calculate:
1. Expected value of reacting with perfect foresight (WS)
2. Expected value of perfect information (EVPI)

Using Gurobi optimizer for solving the optimization problems.
"""

import pandas as pd
import numpy as np
import random
import math
import time
import itertools
import gurobipy as gp
from gurobipy import GRB

# Constants from problem statement
OVERAGE_COST = 0.09  # Cost per liter if truck is too big
UNDERAGE_COST = 0.13  # Cost per liter if truck is too small

# Load the datasets
costs = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/costs.csv')
randomness = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/randomness.csv')

# Clean up the data - assuming first column is the station names/indices
cost_matrix = costs.iloc[:, 1:].values
stations = randomness.iloc[:, 0].values
probabilities = randomness['Probability'].values
mean_demands = randomness['Mean_Demand'].values
std_deviations = randomness['Std_Dev_Demand'].values

num_stations = len(probabilities)
print(f"Loaded data: {num_stations} gas stations")

def generate_scenario(probabilities, mean_demands, std_deviations):
    """Generate a random scenario based on the given distributions"""
    active_stations = []
    demands = np.zeros(len(probabilities))
    
    for i in range(len(probabilities)):
        if random.random() < probabilities[i]:
            # This station needs refueling
            demand = np.random.normal(mean_demands[i], std_deviations[i])
            demands[i] = max(0, demand)  # Ensure non-negative demand
            active_stations.append(i + 1)  # +1 because station indices start at 1 in the cost matrix
    
    return demands, active_stations

def solve_tsp_gurobi(active_stations, cost_matrix):
    """
    Solve the Traveling Salesman Problem (TSP) using Gurobi
    to find the optimal route visiting all active stations.
    """
    if not active_stations:
        return [], 0
    
    # Create a list of all nodes including depot (0)
    nodes = [0] + active_stations
    n = len(nodes)
    
    # Create a dictionary of distances
    dist = {(i, j): cost_matrix[nodes[i], nodes[j]] for i in range(n) for j in range(n) if i != j}
    
    # Create a new model
    m = gp.Model()
    m.setParam('OutputFlag', 0)  # Suppress output
    
    # Create variables
    vars = {}
    for i in range(n):
        for j in range(n):
            if i != j:
                vars[i, j] = m.addVar(obj=dist[i, j], vtype=GRB.BINARY, name=f'x_{i}_{j}')
    
    # Add degree-2 constraint for each node
    for i in range(n):
        m.addConstr(gp.quicksum(vars[i, j] for j in range(n) if j != i) == 1)
        m.addConstr(gp.quicksum(vars[j, i] for j in range(n) if j != i) == 1)
    
    # For smaller problems, add subtour elimination constraints directly
    if n <= 5:
        # Add subtour elimination for all possible subtours
        for size in range(2, n):
            for subset in [list(c) for c in itertools.combinations(range(1, n), size)]:
                m.addConstr(gp.quicksum(vars[i, j] + vars[j, i] for i in subset for j in subset if i < j) <= len(subset) - 1)
        
        # Optimize normally for small problems
        m.optimize()
    else:
        # For larger problems, use a simpler approach
        # First solve without subtour elimination
        m.optimize()
        
        # Then check for subtours and add constraints iteratively
        max_iterations = 100
        iteration = 0
        
        while iteration < max_iterations:
            # Get the solution
            selected = [(i, j) for i in range(n) for j in range(n) if i != j and vars[i, j].x > 0.5]
            
            # Find subtours
            visited = set()
            subtours = []
            
            for i in range(n):
                if i not in visited:
                    # Find the subtour containing i
                    subtour = [i]
                    visited.add(i)
                    current = i
                    
                    while True:
                        # Find next node in the tour
                        next_nodes = [j for i_, j in selected if i_ == current and j not in visited]
                        if not next_nodes:
                            # Check if we can get back to the start of the subtour
                            cycle_next = [j for i_, j in selected if i_ == current and j == subtour[0]]
                            if cycle_next:
                                subtours.append(subtour)
                            break
                        
                        next_node = next_nodes[0]
                        subtour.append(next_node)
                        visited.add(next_node)
                        current = next_node
            
            # If there's only one subtour, we're done
            if len(subtours) <= 1:
                break
            
            # Add subtour elimination constraints
            for subtour in subtours:
                if 0 not in subtour:  # Only add constraint if depot is not in the subtour
                    m.addConstr(gp.quicksum(vars[i, j] for i in subtour for j in subtour if i != j) <= len(subtour) - 1)
            
            # Re-optimize
            m.optimize()
            iteration += 1
    
    # If the problem is infeasible, return empty route
    if m.status == GRB.INFEASIBLE:
        return [], float('inf')
    
    # Extract the route
    tour = []
    edges = []
    for i in range(n):
        for j in range(n):
            if i != j and vars[i, j].x > 0.5:
                edges.append((i, j))
    
    # Convert edges to a route
    if edges:
        # Find the route starting from depot (0)
        current = 0
        route_nodes = [current]
        while len(route_nodes) < n:
            for i, j in edges:
                if i == current and j not in route_nodes:
                    current = j
                    route_nodes.append(current)
                    break
        
        # Remove depot from the route
        route_nodes = route_nodes[1:]
        
        # Convert route indices to actual station numbers
        tour = [nodes[i] for i in route_nodes]
    
    return tour, m.objVal

def calculate_size_mismatch_cost(truck_size, total_demand):
    """Calculate the cost of truck size mismatch"""
    if truck_size >= total_demand:
        # Truck is too big
        return OVERAGE_COST * (truck_size - total_demand)
    else:
        # Truck is too small
        return UNDERAGE_COST * (total_demand - truck_size)

def evaluate_scenario(truck_size, demands, active_stations, cost_matrix):
    """Evaluate total cost for a scenario with given truck size"""
    if not active_stations:
        return 0  # No stations to visit means no cost
    
    # Find optimal route using Gurobi
    route, travel_cost = solve_tsp_gurobi(active_stations, cost_matrix)
    
    # Calculate total demand
    total_demand = sum(demands[station-1] for station in active_stations)  # Adjust index
    
    # Calculate size mismatch cost
    mismatch_cost = calculate_size_mismatch_cost(truck_size, total_demand)
    
    # Total cost
    total_cost = travel_cost + mismatch_cost
    
    return total_cost

def find_optimal_truck_size_gurobi(scenarios, cost_matrix):
    """Find the optimal truck size using Gurobi with fixed formulation"""
    # Create a new model
    m = gp.Model("truck_size_optimization")
    m.setParam('OutputFlag', 0)  # Suppress output
    
    # Create a variable for truck size (continuous)
    truck_size = m.addVar(lb=0.0, name="truck_size")
    
    # Create variables for each scenario's cost
    scenario_costs = []
    for s_idx, (demands, active_stations) in enumerate(scenarios):
        if not active_stations:
            # No active stations means no cost for this scenario
            scenario_costs.append(0)
            continue
            
        # Find optimal route for this scenario
        route, travel_cost = solve_tsp_gurobi(active_stations, cost_matrix)
        
        # Calculate total demand for this scenario
        total_demand = sum(demands[station-1] for station in active_stations)
        
        # Add variables for overage and underage with simpler formulation
        overage = m.addVar(lb=0.0, name=f"overage_{s_idx}")
        underage = m.addVar(lb=0.0, name=f"underage_{s_idx}")
        
        # Define overage and underage directly with proper constraints
        m.addConstr(overage >= truck_size - total_demand)
        m.addConstr(underage >= total_demand - truck_size)
        
        # Cost for this scenario
        scenario_cost = travel_cost + OVERAGE_COST * overage + UNDERAGE_COST * underage
        scenario_costs.append(scenario_cost)
    
    # Set objective to minimize average cost across all scenarios
    m.setObjective(gp.quicksum(scenario_costs) / len(scenarios), GRB.MINIMIZE)
    
    # Optimize
    m.optimize()
    
    # Return optimal truck size and objective value
    if m.status == GRB.OPTIMAL:
        return truck_size.x, m.objVal
    else:
        return None, float('inf')

def wait_and_see_cost(demands, active_stations, cost_matrix):
    """Calculate the optimal cost with perfect information for a scenario"""
    if not active_stations:
        return 0  # No stations to visit
    
    # Find optimal route using Gurobi
    route, travel_cost = solve_tsp_gurobi(active_stations, cost_matrix)
    
    # With perfect information, we know exactly how much fuel is needed
    total_demand = sum(demands[station-1] for station in active_stations)
    
    # The optimal truck size is exactly the total demand
    optimal_truck_size = total_demand
    
    # With perfect sizing, there's no mismatch cost
    mismatch_cost = 0  # or calculate_size_mismatch_cost(optimal_truck_size, total_demand) which should be 0
    
    # Total cost is just the travel cost (no mismatch)
    total_cost = travel_cost + mismatch_cost
    
    return total_cost

def run_saa_trial(num_scenarios, probabilities, mean_demands, std_deviations, cost_matrix):
    """Run one SAA trial and calculate both optimal cost and wait-and-see cost"""
    # Generate scenarios
    scenarios = []
    for _ in range(num_scenarios):
        demands, active_stations = generate_scenario(probabilities, mean_demands, std_deviations)
        scenarios.append((demands, active_stations))
    
    # Find optimal truck size using Gurobi
    optimal_truck_size, optimal_cost = find_optimal_truck_size_gurobi(scenarios, cost_matrix)
    
    # Calculate wait-and-see cost for each scenario
    ws_costs = []
    for demands, active_stations in scenarios:
        ws_cost = wait_and_see_cost(demands, active_stations, cost_matrix)
        ws_costs.append(ws_cost)
    
    # Average wait-and-see cost
    avg_ws_cost = np.mean(ws_costs)
    
    return optimal_truck_size, optimal_cost, avg_ws_cost

def main():
    """Main function to solve part (f)"""
    start_time = time.time()
    
    # Parameters for SAA
    num_trials = 20
    num_scenarios = 10
    
    # Run SAA trials
    truck_sizes = []
    objective_values = []
    ws_values = []
    
    print(f"Running {num_trials} trials with {num_scenarios} scenarios per trial...")
    
    for trial in range(num_trials):
        optimal_truck_size, optimal_cost, ws_cost = run_saa_trial(
            num_scenarios, probabilities, mean_demands, std_deviations, cost_matrix
        )
        
        truck_sizes.append(optimal_truck_size)
        objective_values.append(optimal_cost)
        ws_values.append(ws_cost)
        
        print(f"Trial {trial+1}:")
        print(f"  Optimal truck size = {optimal_truck_size:.2f}")
        print(f"  Recourse problem cost = ${optimal_cost:.2f}")
        print(f"  Wait-and-see cost = ${ws_cost:.2f}")
    
    # Calculate average values
    avg_objective = np.mean(objective_values)
    avg_ws = np.mean(ws_values)
    
    # Calculate EVPI
    evpi = avg_objective - avg_ws
    
    # Calculate statistics
    std_dev_objective = np.std(objective_values, ddof=1)
    std_error_objective = std_dev_objective / math.sqrt(num_trials)
    
    std_dev_ws = np.std(ws_values, ddof=1)
    std_error_ws = std_dev_ws / math.sqrt(num_trials)
    
    std_dev_evpi = np.std([obj - ws for obj, ws in zip(objective_values, ws_values)], ddof=1)
    std_error_evpi = std_dev_evpi / math.sqrt(num_trials)
    
    # Calculate confidence intervals for EVPI
    ci_90_evpi = (evpi - 1.645 * std_error_evpi, evpi + 1.645 * std_error_evpi)
    ci_95_evpi = (evpi - 1.96 * std_error_evpi, evpi + 1.96 * std_error_evpi)
    
    # Print results
    print("\n=== RESULTS FOR PART (f) ===")
    print(f"Average optimal truck size: {np.mean(truck_sizes):.2f} liters")
    print(f"Expected cost of the stochastic solution: ${avg_objective:.2f}")
    print(f"Wait-and-See value (WS): ${avg_ws:.2f}")
    print(f"Expected Value of Perfect Information (EVPI): ${evpi:.2f}")
    print(f"90% confidence interval for EVPI: (${ci_90_evpi[0]:.2f}, ${ci_90_evpi[1]:.2f})")
    print(f"95% confidence interval for EVPI: (${ci_95_evpi[0]:.2f}, ${ci_95_evpi[1]:.2f})")
    
    print(f"\nComputation time: {time.time() - start_time:.2f} seconds")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        print(traceback.format_exc())

Loaded data: 14 gas stations
Running 20 trials with 10 scenarios per trial...
Trial 1:
  Optimal truck size = 2568.89
  Recourse problem cost = $151.53
  Wait-and-see cost = $120.80
Trial 2:
  Optimal truck size = 2600.72
  Recourse problem cost = $179.39
  Wait-and-see cost = $119.60
Trial 3:
  Optimal truck size = 2544.21
  Recourse problem cost = $167.44
  Wait-and-see cost = $117.40
Trial 4:
  Optimal truck size = 2168.86
  Recourse problem cost = $154.59
  Wait-and-see cost = $118.30
Trial 5:
  Optimal truck size = 2218.05
  Recourse problem cost = $162.00
  Wait-and-see cost = $108.60
Trial 6:
  Optimal truck size = 2857.54
  Recourse problem cost = $155.37
  Wait-and-see cost = $112.10
Trial 7:
  Optimal truck size = 2436.15
  Recourse problem cost = $145.52
  Wait-and-see cost = $123.70
Trial 8:
  Optimal truck size = 2309.64
  Recourse problem cost = $164.29
  Wait-and-see cost = $121.80
Trial 9:
  Optimal truck size = 2605.88
  Recourse problem cost = $146.48
  Wait-and-see c

2g claude